In [1]:
import os
from itertools import permutations 
import sys
import numpy as np
import scipy.sparse as sp
from tqdm import tqdm
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS

In [3]:
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits, evaluate_preds
from ggcn import GaussianGraphConvolution, kl_reg

# Data

In [4]:
def attack_edges(attack_ratio):
    def _attack_edges(edges):
        all_edges = set(permutations(list(range(edges.max())), 2))
        edges = set(map(tuple, edges))
        sample_edges = all_edges - edges
        attack_edges = np.random.choice(list(range(len(sample_edges))), 
            size=int(len(edges) * attack_ratio), 
            replace=False)
        attack_edges = [edge for (i, edge) in enumerate(list(sample_edges)) if i in attack_edges]
        edges = list(edges) + attack_edges
        return np.array(edges)
    return _attack_edges

In [5]:
X, A, y = load_data('data/cora/', dataset='cora', attack_edges=attack_edges(0))
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [6]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [7]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [8]:
y

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=int32)

In [9]:
graph = [X, A]

In [10]:
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True, name='graph')]
G

In [11]:
X_in = Input(shape=(X.shape[1], ), name='features')
X_in

<tf.Tensor 'features:0' shape=(?, 1433) dtype=float32>

# Model

## Training Loop

In [12]:
def train(model, epochs=200, patience=10):
    wait = 0
    preds = None
    best_val_loss = 99999
    # Fit
    progress = tqdm(range(1, epochs + 1))
    for epoch in progress:
        # Single training iteration (we mask nodes without labels for loss calculation)
        model.fit(graph, y_train, sample_weight=train_mask,
                  batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)
        # Predict on full dataset
        preds = model.predict(graph, batch_size=A.shape[0])
        # Train / validation scores
        train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val], [idx_train, idx_val])
        # update the progress bar
        progress.set_postfix(
#             train_loss=train_val_loss[0],
            train_acc=train_val_acc[0],
#             val_loss=train_val_loss[1],
            val_acc=train_val_acc[1],
        )
        # Early stopping
        if train_val_loss[1] < best_val_loss:
            best_val_loss = train_val_loss[1]
            wait = 0
        else:
            if wait >= patience:
                break
            wait += 1
    # return test set evaluation metrics
    return evaluate_preds(preds, [y_test], [idx_test])

## GCN (baseline)

In [13]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(32, 1, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], 1, activation='softmax')([H]+G)
gcn = Model(inputs=[X_in]+G, outputs=Y)
gcn.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))
gcn.summary()



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
features (InputLayer)           (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1433)         0           features[0][0]                   
__________________________________________________________________________________________________
graph (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 32)           45888       dropout_1[0][0]         

In [14]:
train(gcn)[1]

  0%|          | 0/200 [00:00<?, ?it/s]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


100%|██████████| 200/200 [00:16<00:00, 12.00it/s, train_acc=0.986, val_acc=0.83] 


[0.816]

## Gaussian Graph Convolution

In [15]:
B1 = 5e-4
B2 = 5e-4

In [18]:
H = Dropout(0.6)(X_in)
H1 = GaussianGraphConvolution(16,
    is_first=True,
    mean_activation='elu',
    mean_regularizer=l2(B1),
    variance_regularizer=l2(B1),
    variance_activation='relu',
    dropout=0.6
)([H]+G)
Y = GaussianGraphConvolution(y.shape[1],
    is_last=True,
    mean_activation='elu',
    mean_regularizer=l2(B1),
    variance_regularizer=l2(B1),
    variance_activation='relu',
    last_activation='softmax',
)(H1+G)
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))
model.add_loss(kl_reg(*H1, B2), H1)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
features (InputLayer)           (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 1433)         0           features[0][0]                   
__________________________________________________________________________________________________
graph (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
gaussian_graph_convolution_3 (G [(None, 16), (None,  45856       dropout_4[0][0]                  
                                                                 graph[0][0]                      
__________

In [19]:
train(model)[1]


 26%|██▌       | 51/200 [00:06<00:18,  7.98it/s, train_acc=0.793, val_acc=0.717]


 50%|█████     | 101/200 [00:12<00:10,  9.36it/s, train_acc=0.914, val_acc=0.81] 


 80%|███████▉  | 159/200 [00:18<00:04,  9.08it/s, train_acc=0.929, val_acc=0.783]


 92%|█████████▏| 183/200 [00:21<00:01,  9.47it/s, train_acc=0.957, val_acc=0.81] 

[0.808]